# Samples: Load Balancing

Sets up an APIM instance that demonstrates load balancing and circuit breaking across backends. 

⚙️ **Supported infrastructures**: apim-aca, afd-apim (with ACA)

⌚ **Expected *Run All* runtime (excl. infrastructure prerequisite): ~[NOTEBOOK RUNTIME] minute**

## Objectives

1. Understand how backends can be configured to balance load in a prioritized, weighted manner.
1. Learn how circuit breakers aid with load balancing.

## Lab Components

[DESCRIBE IN MORE DETAIL WHAT THIS LAB SETS UP AND HOW THIS BENEFITS THE LEARNER/USER.]

## Configuration

1. Decide which of the [Infrastructure Architectures](../../README.md#infrastructure-architectures) you wish to use.
    1. If the infrastructure _does not_ yet exist, navigate to the desired [infrastructure](../../infrastructure/) folder and follow its README.md.
    1. If the infrastructure _does_ exist, adjust the `user-defined parameters` in the _Initialize notebook variables_ below. Please ensure that all parameters match your infrastructure.

### Initialize notebook variables

Configures everything that's needed for deployment. 

[ADD ANY SPECIAL INSTRUCTIONS]

**Modify entries under _1) User-defined parameters_ and _3) Define the APIs and their operations and policies_**.

In [7]:
import utils
from apimtypes import *

# 1) User-defined parameters (change these as needed)
rg_location = 'eastus2'
index       = 1
deployment  = INFRASTRUCTURE.APIM_ACA
tags        = ['load-balancing']       # [ENTER DESCRIPTIVE TAG(S)]
api_prefix  = 'lb-'                    # OPTIONAL: ENTER A PREFIX FOR THE APIS TO REDUCE COLLISION POTENTIAL WITH OTHER SAMPLES

# 2) Service-defined parameters (please do not change these)
rg_name = utils.get_infra_rg_name(deployment, index)

# 3) Define the APIs and their operations and policies

# Policies
aca_backend_pool_prioritized_policy_xml = utils.read_policy_xml('./aca-backend-pool-prioritized.xml')
aca_backend_pool_weighted_policy_xml    = utils.read_policy_xml('./aca-backend-pool-weighted.xml')

# Prioritized ACA Backend Pool
aca_priority_get = GET_APIOperation('This is a GET for the prioritized backend pool API')
aca_priority = API(f'{api_prefix}prioritized-aca-pool', 'Prioritized backend pool', f'/{api_prefix}prioritized', 'This is the API for the prioritized backend pool.', policyXml = aca_backend_pool_prioritized_policy_xml, operations = [aca_priority_get], tags = tags)

# Prioritized ACA Backend Pool
aca_weighted_get = GET_APIOperation('This is a GET for the weighted backend pool API')
aca_weighted = API(f'{api_prefix}weighted-aca-pool', 'Weighted backend pool', f'/{api_prefix}weighted', 'This is the API for the weighted backend pool.', policyXml = aca_backend_pool_weighted_policy_xml, operations = [aca_weighted_get], tags = tags)

# APIs Array
apis: List[API] = [aca_priority, aca_weighted]

utils.print_ok('Notebook initialized')

👉🏽 Resource group name      : apim-infra-apim-aca-1

✅ Notebook initialized ⌚ 10:50:12.035954


### Create deployment using Bicep

Creates the bicep deployment into the previously-specified resource group. A bicep parameters file will be created prior to execution.

In [8]:
import utils

# 1) Define the Bicep parameters with serialized APIs
bicep_parameters = {
    'apis': {'value': [api.to_dict() for api in apis]}
}

# 2) Infrastructure must be in place before samples can be layered on top
if not utils.does_resource_group_exist(rg_name):
    utils.print_error(f'The specified infrastructure resource group and its resources must exist first. Please check that the user-defined parameters above are correctly referencing an existing infrastructure. If it does not yet exist, run the desired infrastructure in the /infra/ folder first.')
    raise SystemExit(1)

# 3) Run the deployment
output = utils.create_bicep_deployment_group(rg_name, rg_location, deployment, bicep_parameters)

# 4) Print a deployment summary, if successful; otherwise, exit with an error
if not output.success:
    raise SystemExit('Deployment failed')

if output.success and output.json_data:
    apim_gateway_url = output.get('apimResourceGatewayURL', 'APIM API Gateway URL')

utils.print_ok('Deployment completed')

⚙️ az group show --name apim-infra-apim-aca-1
⚙️ az group show --name apim-infra-apim-aca-1
📝 Updated the policy XML in the bicep parameters file 'params.json'
⚙️ az deployment group create --name apim-aca --resource-group apim-infra-apim-aca-1 --template-file main.bicep --parameters params.json --query
"properties.outputs"
👉🏽 APIM API Gateway URL     : https://apim-iwk4oefl2gc5a.azure-api.net

✅ Deployment completed ⌚ 10:51:29.205384


### Verify API Request Success

Assert that the deployment was successful by making simple calls to APIM. 

❗️ If the infrastructure shields APIM and requires a different ingress (e.g. Azure Front Door), the request to the APIM gateway URl will fail by design. Obtain the Front Door endpoint hostname and try that instead.

In [9]:
# import utils
# from apimrequests import ApimRequests

# [ADD RELEVANT TESTS HERE]

# # 1) Issue a direct request to API Management
# # reqsApim = ApimRequests(apim_gateway_url)
# # reqsApim.singleGet('/request-headers', msg = 'Calling Request Headers API via API Management Gateway URL. Response codes 200 and 403 are both valid depending on the infrastructure used.')

# # # 2) Issue requests against Front Door.
# # # Check if the infrastructure architecture deployment uses Azure Front Door.
# # utils.print_message('Checking if the infrastructure architecture deployment uses Azure Front Door.', blank_above = True)
# # afd_endpoint_url = utils.get_frontdoor_url(deployment, rg_name)

# # if afd_endpoint_url:
# #     reqsAfd = ApimRequests(afd_endpoint_url)
# #     reqsAfd.singleGet('/request-headers', msg = 'Calling Request Headers API via via Azure Front Door. Expect 200.')

# utils.print_ok('All done!')